In [1]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
import pyte_visa_utils as pyte
import pyte16

In [2]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt
from instFunction import *

In [3]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.60::5025::SOCKET'

inst = pyte.open_session(inst_addr)
resp = inst.query("*IDN?")
print('connected to: ' + resp)

Failed to open "TCPIP::192.168.0.60::5025::SOCKET"
(<class 'pyvisa.errors.VisaIOError'>, VisaIOError('VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.'), <traceback object at 0x0000028910819688>)


AttributeError: 'NoneType' object has no attribute 'query'

In [7]:
# Paranoia Level (0: low, 1: normal, 2: debug)
paranoia_level = 2

# Reset the instrument
pyte.send_cmd(inst, '*CLS; *RST', paranoia_level)

In [8]:
# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
dac_mode = resp
print('DAC mode {0} '.format(dac_mode))

if dac_mode == 'M0':
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac // 2.0

# Get number of channels
resp = inst.query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))



DAC-Mode: 8
Number of channels: 2
Max segment number: 65536
Available memory per DDR: 4,286,578,688 wave-bytes


In [9]:
seglen=102400000
num_cycles=2000
cyclelen = seglen / num_cycles
#sin_wav=pyte.build_sine_wave(cycle_len,num_cycles)
x = np.linspace(start=0, stop=2 * np.pi * num_cycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
if dac_mode == 'M0':
    wave = y.astype(np.uint16)
else:
    wave = y.astype(np.uint8)


In [10]:
# data-type code: 'B' = uint8, 'H' = uint16
data_type = 'H' if dac_mode == 'M0' else 'B'


channb=1
segnum=1


cmd = ':INST:CHAN {0}'.format(channb)
pyte.send_cmd(inst, cmd, paranoia_level)
    
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
pyte.send_cmd(inst, cmd, paranoia_level)

cmd = ':TRAC:SEL {0}'.format(segnum)
pyte.send_cmd(inst, cmd, paranoia_level)

inst.timeout = 30000

#start_time = time.time()
tic = time.perf_counter()

inst.write_binary_values('*OPC?; :TRAC:DATA', wave, datatype=data_type)

#end_time = time.time()
#time_lapsed = end_time - start_time

#print('Downloaded the wave in "{0}" seconds'.format(time_lapsed))

toc = time.perf_counter()
timer = float(toc-tic)

print('Downloaded the wave in "{0}" seconds'.format(timer))

resp = inst.read();
inst.timeout = 10000;
    
resp = inst.query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{0}" after writing binary values'.format(resp))

  

Downloaded the wave in "12.610927299999958" seconds
